# Lab 2 - Travelling Salesman Problem (TSP)

In [437]:
import pandas as pd
import numpy as np
import geopy.distance
from tqdm.auto import tqdm
from dataclasses import dataclass

In [438]:
PATH = "cities/"
INSTANCE = "china.csv"

In [439]:
class City:
    
    @staticmethod
    def distance(start, end):
        return geopy.distance.geodesic(
            (start.lat, start.lon), (end.lat, end.lon)
        ).km
    
    def __init__(self, name, lat, lon):
        self.name: str = name
        self.lat: float | np.float64 = lat
        self.lon: float | np.float64 = lon
        
    def __repr__(self):
        return f"{self.name}"
    
    def __str__(self):
        return f"{self.name} ({self.lat}°, {self.lon}°)"

In [440]:
cities = pd.read_csv(f"{PATH}{INSTANCE}", header=None, names=["name", "lat", "lon"])
cities, type(cities)

(         name        lat         lon
 0      Acheng  45.540000  126.960000
 1        Aksu  41.150000   80.250000
 2       Alaer  40.515556   81.263611
 3       Altay  47.840000   88.130000
 4        Anbu  23.460000  116.680000
 ..        ...        ...         ...
 721    Ziyang  30.140000  104.640000
 722  Zoucheng  35.400000  116.966667
 723   Zouxian  35.410000  116.940000
 724    Zunhua  40.183333  117.966667
 725     Zunyi  27.700000  106.920000
 
 [726 rows x 3 columns],
 pandas.core.frame.DataFrame)

In [441]:
coordinates = np.array([City(city.name, city.lat, city.lon) for city in cities.itertuples()])
coordinates

array([Acheng, Aksu, Alaer, Altay, Anbu, Anda, Anguo, Ankang, Anlu,
       Anning, Anqing, Anqiu, Anshan, Anshun, Anyang, Atushi, Baicheng,
       Baise, Baishan, Baiyin, Bantou, Baoding, Baoji, Baoshan, Baotou,
       Bayannaoer, Bazhong, Bazhou, Beian, Beibei, Beihai, Beijing,
       Beiliu, Beining, Beipiao, Bengbu, Benxi, Bijie, Binzhou,
       Bole County, Boshan, Botou, Bozhou, Buhe, Cangzhou, Cenxi,
       Changchun, Changde, Changge, Changji, Changle, Changning,
       Changping, Changsha, Changshu, Changyi, Changzhi, Changzhou,
       Chaohu, Chaoyang, Chaozhou, Chengde, Chengdu, Chenghai, Chenzhou,
       Chibi, Chifeng, Chishui, Chizhou, Chongqing, Chongzhou, Chongzuo,
       Chuxiong, Chuzhou, Cixi, Conghua, Daan, Dafeng, Dali, Dalian,
       Daliang, Dandong, Dangyang, Danjiangkou, Danshui, Danyang, Danzhou,
       Dashiqiao, Datong, Dawukou, Daxian, Daye County, Dazhou, Dehui,
       Dengfeng, Dengta, Dengzhou, Deqing, Dexing, Deyang, Dezhou, Didao,
       Dingxi, Dingzho

## Helper functions

In [442]:
def distance_matrix(coordinates: list) -> np.ndarray:
    num_cities = len(coordinates)
    dist_matrix = np.zeros((num_cities, num_cities))
    
    for i in tqdm(range(num_cities)):
        for j in range(i+1):
            dist_matrix[i, j] = dist_matrix[j, i] = City.distance(coordinates[i], coordinates[j]) if i != j else 0
          
    return dist_matrix


def cost(solution: np.ndarray, dist_matrix: np.ndarray) -> np.float64 | float:
    """Cost of a cycle"""
    return np.sum(
        np.array([
            dist_matrix[start, end] for (start, end) in zip(solution[:-1], solution[1:])
        ])
    )

## Greedy

In [443]:
def greedy_solve(coordinates, dist_matrix):
    """Greedy algorithm with random initialization: sub-optimal"""
    temp = dist_matrix.copy()
    
    num_cities = len(coordinates)
    visited = np.full(num_cities, False)
    start_index = np.random.randint(0, num_cities)
    
    solution = -np.ones(num_cities+1, dtype=np.int16)
    solution[0], visited[0] = start_index, True
    for step in tqdm(range(num_cities-1)):
        temp[:, start_index] = np.inf
        next_index = np.argmin(temp[start_index])
        
        solution[step+1] = next_index
        start_index = next_index
        visited[start_index] = True
        
    solution[-1] = solution[0]
    
    return solution

In [444]:
dist_matrix = distance_matrix(coordinates)
solution = greedy_solve(coordinates, dist_matrix)
for (start, end) in zip(solution[:-1], solution[1:]):
    print(f"{coordinates[start]} -> {coordinates[end]}, km: {dist_matrix[start, end]:.3f}")
print()

cost_sol = cost(solution, dist_matrix)
print(f"Total cost: {cost_sol:.3f} km")

  0%|          | 0/726 [00:00<?, ?it/s]

  0%|          | 0/725 [00:00<?, ?it/s]

Gaoyao (23.030767°, 112.450217°) -> Zhaoqing (23.05°, 112.45°), km: 2.130
Zhaoqing (23.05°, 112.45°) -> Sihui (23.35°, 112.683333°), km: 40.919
Sihui (23.35°, 112.683333°) -> Xinan (23.17°, 112.89°), km: 29.062
Xinan (23.17°, 112.89°) -> Foshan (23.03°, 113.12°), km: 28.206
Foshan (23.03°, 113.12°) -> Guangzhou (23.12°, 113.25°), km: 16.637
Guangzhou (23.12°, 113.25°) -> Huangpu (23.1°, 113.42°), km: 17.555
Huangpu (23.1°, 113.42°) -> Shiqiao (22.94°, 113.36°), km: 18.756
Shiqiao (22.94°, 113.36°) -> Daliang (22.84°, 113.25°), km: 15.812
Daliang (22.84°, 113.25°) -> Xiaolan (22.78°, 113.28°), km: 7.324
Xiaolan (22.78°, 113.28°) -> Zhongshan (22.53°, 113.35°), km: 28.604
Zhongshan (22.53°, 113.35°) -> Jiangmen (22.58°, 113.08°), km: 28.318
Jiangmen (22.58°, 113.08°) -> Shaping (22.77°, 112.96°), km: 24.388
Shaping (22.77°, 112.96°) -> Kaiping (22.369722°, 112.684444°), km: 52.611
Kaiping (22.369722°, 112.684444°) -> Taicheng (22.26°, 112.77°), km: 15.011
Taicheng (22.26°, 112.77°) -> Ta